### Architecture 체크

In [1]:
import sys
import os

from dynamic_network_architectures.architectures.unet import PlainConvUNet
from dynamic_network_architectures.architectures.DANN_unet import DANNConvUNet

In [2]:
import torch
from torch import nn
from dynamic_network_architectures.architectures.unet import PlainConvUNet

def test_plain_conv_unet():
    # Model parameters from 3d_fullres configuration
    input_channels = 1
    n_stages = 6
    features_per_stage = [32, 64, 128, 256, 320, 320]
    conv_op = nn.Conv3d
    kernel_sizes = [
        [3, 3, 3],
        [3, 3, 3],
        [3, 3, 3],
        [3, 3, 3],
        [3, 3, 3],
        [3, 3, 3]
    ]
    strides = [
        [1, 1, 1],
        [2, 2, 2],
        [2, 2, 2],
        [2, 2, 2],
        [2, 2, 2],
        [1, 2, 2]
    ]
    n_conv_per_stage = [2, 2, 2, 2, 2, 2]
    num_classes = 4
    n_conv_per_stage_decoder = [2, 2, 2, 2, 2]
    conv_bias = True
    norm_op = nn.InstanceNorm3d
    norm_op_kwargs = {"eps": 1e-5, "affine": True}
    dropout_op = None  # null in the configuration
    dropout_op_kwargs = None
    nonlin = nn.LeakyReLU
    nonlin_kwargs = {"inplace": True}
    deep_supervision = False  # 유지
    nonlin_first = False  # 유지

    # Model instantiation
    model = PlainConvUNet(
        input_channels,
        n_stages,
        features_per_stage,
        conv_op,
        kernel_sizes,
        strides,
        n_conv_per_stage,
        num_classes,
        n_conv_per_stage_decoder,
        conv_bias,
        norm_op,
        norm_op_kwargs,
        dropout_op,
        dropout_op_kwargs,
        nonlin,
        nonlin_kwargs,
        deep_supervision,
        nonlin_first
    )
    print("PlainConvUNet 모델 생성 성공!")

    # Testing input
    input_shape = (2, input_channels, 96, 160, 160)  # batch_size=2 from configuration
    input_tensor = torch.randn(input_shape)

    # Forward pass
    output = model(input_tensor)

    # Output shape
    print(f"Input shape: {input_tensor.shape}")
    print(f"Output shape: {output.shape}")

if __name__ == "__main__":
    test_plain_conv_unet()


PlainConvUNet 모델 생성 성공!
Input shape: torch.Size([2, 1, 96, 160, 160])
Output shape: torch.Size([2, 4, 96, 160, 160])


DANNConvUNet 모델 생성 성공!
6
10
10


RuntimeError: mat1 and mat2 shapes cannot be multiplied (2x48000 and 192000x2)

In [1]:
import torch
from torch import nn
import numpy as np
from typing import Union, Type, List, Tuple
# additional import
from dynamic_network_architectures.building_blocks.CBAM_blocks import CBAM
from dynamic_network_architectures.architectures.unet import UNetDecoder
from dynamic_network_architectures.building_blocks.CBAMPlainConvEncoder import CBAMPlainConvEncoder
from dynamic_network_architectures.building_blocks.residual_encoders import ResidualEncoder
from dynamic_network_architectures.building_blocks.plain_conv_encoder import PlainConvEncoder
from dynamic_network_architectures.building_blocks.CBAMUNetDecoder import CBAMUNetDecoder

import importlib
from dynamic_network_architectures.architectures import cbam_unet
importlib.reload(cbam_unet)
from dynamic_network_architectures.architectures.cbam_unet import CBAMUNet



def test_cbam_unet_decoder():
    encoder = PlainConvEncoder(
        input_channels=3,
        n_stages=4,
        features_per_stage=[64, 128, 256, 512],
        conv_op=nn.Conv3d,
        kernel_sizes=[3] * 4,
        strides=[2] * 4,
        n_conv_per_stage=2,
        conv_bias=True,
        norm_op=nn.BatchNorm3d,
        norm_op_kwargs={"eps": 1e-5, "affine": True},
        dropout_op=None,
        dropout_op_kwargs=None,
        nonlin=nn.ReLU,
        nonlin_kwargs={"inplace": True},
        return_skips=True
    )

    decoder = CBAMUNetDecoder(
        encoder=encoder,
        num_classes=4,
        n_conv_per_stage=2,
        deep_supervision=False,
        norm_op=nn.BatchNorm3d,
        norm_op_kwargs={"eps": 1e-5, "affine": True},
        dropout_op=None,
        dropout_op_kwargs=None,
        nonlin=nn.ReLU,
        nonlin_kwargs={"inplace": True},
        conv_bias=True
    )

    x = torch.randn(1, 3, 128, 128, 128)
    skips = encoder(x)
    output = decoder(skips)

    print(f"Decoder output shape: {output.shape}")


test_cbam_unet_decoder()


Decoder output shape: torch.Size([1, 4, 64, 64, 64])


In [ ]:
import torch
from torch import nn
from dynamic_network_architectures.architectures.DANN_unet import DANNConvUNet
from batchgenerators.utilities.file_and_folder_operations import join, isfile, load_json
from nnunetv2.utilities.plans_handling.plans_handler import PlansManager

preprocessed_dataset_folder = '/home/seongwoo/dataset/nnunetFrame/nnunet_preprocessed/Dataset301_Kits19All2'
plans_identifier = 'DANNPlans'
preprocessed_dataset_folder_base = preprocessed_dataset_folder
plans_file = join(preprocessed_dataset_folder_base, plans_identifier + '.json')
plans = load_json(plans_file)
dataset_json = load_json(join(preprocessed_dataset_folder_base, 'dataset.json'))
plans_manager = PlansManager(plans)
label_manager = plans_manager.get_label_manager(dataset_json)
configuration = '3d_fullres'
configuration_manager = plans_manager.get_configuration(configuration)

def test_cbam_unet():
    # Model parameters from 3d_fullres configuration
    input_channels = 1  # CT 데이터라 가정
    n_stages = 6
    features_per_stage = [32, 64, 128, 256, 320, 320]
    conv_op = nn.Conv3d
    kernel_sizes = [
        [3, 3, 3],
        [3, 3, 3],
        [3, 3, 3],
        [3, 3, 3],
        [3, 3, 3],
        [3, 3, 3]
    ]
    strides = [
        [1, 1, 1],
        [2, 2, 2],
        [2, 2, 2],
        [2, 2, 2],
        [2, 2, 2],
        [1, 2, 2]
    ]
    n_conv_per_stage = [2, 2, 2, 2, 2, 2]
    num_classes = 4
    n_conv_per_stage_decoder = [2, 2, 2, 2, 2]
    conv_bias = True
    norm_op = nn.InstanceNorm3d
    norm_op_kwargs = {"eps": 1e-5, "affine": True}
    dropout_op = None
    dropout_op_kwargs = None
    nonlin = nn.LeakyReLU
    nonlin_kwargs = {"inplace": True}
    deep_supervision = False
    nonlin_first = False

    # Model instantiation
    model = DANNConvUNet(
        input_channels,
        n_stages,
        features_per_stage,
        conv_op,
        kernel_sizes,
        strides,
        n_conv_per_stage,
        num_classes,
        n_conv_per_stage_decoder,
        conv_bias,
        norm_op,
        norm_op_kwargs,
        dropout_op,
        dropout_op_kwargs,
        nonlin,
        nonlin_kwargs,
        deep_supervision,
        nonlin_first
    )
    print("DANNConvUNet 모델 생성 성공!")

    # Testing input
    input_shape = (2, input_channels, 96, 160, 160)  # batch_size=2 from configuration
    input_tensor = torch.randn(input_shape)

    # Forward pass
    model.make_classifier((input_shape[2],input_shape[3],input_shape[4]), features_per_stage, 2)
    output = model(input_tensor)

    # Output shape
    print(f"Input shape: {input_tensor.shape}")
    print(f"Output shape: {output.shape}")

if __name__ == "__main__":
    test_cbam_unet()


### DataLoader 체크

In [ ]:
from batchgenerators.dataloading.nondet_multi_threaded_augmenter import NonDetMultiThreadedAugmenter
from nnunetv2.training.dataloading.data_loader_2d import nnUNetDataLoader2D
from nnunetv2.training.dataloading.dann_data_loader_3d import DANNDataLoader3D
from nnunetv2.training.dataloading.nnunet_dataset import nnUNetDataset
from nnunetv2.utilities.plans_handling.plans_handler import PlansManager
from batchgenerators.utilities.file_and_folder_operations import join, isfile, load_json
from nnunetv2.utilities.dataset_name_id_conversion import maybe_convert_to_dataset_name

preprocessed_dataset_folder = '/home/seongwoo/dataset/nnunetFrame/nnunet_preprocessed/Dataset300_Kits19NCCT'
batch_size = 2
patch_size = (96, 160, 160)
initial_patch_size = get_initial_patch_size(patch_size)

plans_identifier = 'nnUNetPlans'
preprocessed_dataset_folder_base = preprocessed_dataset_folder
plans_file = join(preprocessed_dataset_folder_base, plans_identifier + '.json')
plans = load_json(plans_file)
dataset_json = load_json(join(preprocessed_dataset_folder_base, 'dataset.json'))

plans_manager = PlansManager(plans)
label_manager = plans_manager.get_label_manager(dataset_json)

dataset_tr = nnUNetDataset(preprocessed_dataset_folder, None,
                                   folder_with_segs_from_previous_stage=None,
                                   num_images_properties_loading_threshold=0)

# training pipeline
tr_transforms = self.get_training_transforms(
    patch_size, rotation_for_DA, deep_supervision_scales, mirror_axes, do_dummy_2d_data_aug,
    use_mask_for_norm=self.configuration_manager.use_mask_for_norm,
    is_cascaded=self.is_cascaded, foreground_labels=self.label_manager.foreground_labels,
    regions=self.label_manager.foreground_regions if self.label_manager.has_regions else None,
    ignore_label=self.label_manager.ignore_label)


dl_tr = DANNDataLoader3D(dataset_tr, batch_size,
                                       initial_patch_size,
                                       patch_size,
                                       label_manager,
                                       oversample_foreground_percent=0.33,
                                       sampling_probabilities=None, pad_sides=None, transforms=tr_transforms)